# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-4.041100047703289

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0013 s/step ...done
MCMC, adapting ϵ (25 steps)
1.7e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (100 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
4.3e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
7.6e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
9.4e-6 s/step ...done
MCMC (1000 steps)
6.4e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.39607], -3.13652, 2, DoubledTurn, 0.956733, 3), NUTS_Transition{Array{Float64,1},Float64}([0.812781], -3.46896, 1, DoubledTurn, 0.817852, 1), NUTS_Transition{Array{Float64,1},Float64}([1.70656], -3.3517, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.76507], -3.07928, 1, DoubledTurn, 0.993297, 1), NUTS_Transition{Array{Float64,1},Float64}([3.62022], -5.55881, 2, DoubledTurn, 0.651238, 3), NUTS_Transition{Array{Float64,1},Float64}([3.15916], -6.34256, 2, DoubledTurn, 0.980942, 3), NUTS_Transition{Array{Float64,1},Float64}([3.21866], -4.91616, 1, DoubledTurn, 0.990967, 1), NUTS_Transition{Array{Float64,1},Float64}([3.87514], -5.79467, 1, DoubledTurn, 0.91885, 1), NUTS_Transition{Array{Float64,1},Float64}([3.92311], -6.19456, 1, DoubledTurn, 0.995745, 1), NUTS_Transition{Array{Float64,1},Float64}([2.27835], -5.82438, 2, DoubledTurn, 0.999684, 3)  …  NUTS_Transition{Array{Float64

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

338.7967148254395

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.86, min/25%/median/75%/max: 0.0 0.83 0.96 1.0 1.0
  termination: AdjacentTurn => 20% DoubledTurn => 80%
  depth: 1 => 60% 2 => 40%


check the mean

In [11]:
mean(posterior_α)

0.8184584953479987

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*